In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from haversine import haversine, Unit
import itertools
import datetime
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)

### Read in and Clean DFS

In [2]:
daily_lead_df = pd.read_csv('../input_data/daily_lead_80_20.csv')

In [3]:
daily_lead_df['point'] = [(x, y) for x,y in zip(daily_lead_df['latitude'], daily_lead_df['longitude'])]

In [4]:
daily_lead_df['date'] = pd.to_datetime(daily_lead_df['date1'])

In [5]:
unique_lead_df = pd.read_pickle("../../21_6_7/notebooks/daily_lead_unique.pkl")

In [6]:
stag_df = pd.read_stata('../input_data/air_stagnation_73_20.dta')

In [7]:
data = pd.io.stata.read_stata('../input_data/air_stagnation_73_20.dta')
data.to_csv('my_stata_file.csv')
air_stagnation = pd.read_csv('my_stata_file.csv')

In [8]:
air_stagnation = air_stagnation.drop(columns='Unnamed: 0',axis=1)

In [9]:
air_stagnation['cell_point'] = [(x, y) for x,y in zip(air_stagnation['lat'], air_stagnation['long'])]

In [10]:
filt_lead_df = unique_lead_df[['Pb_mean', 'monitorID', 'year','month', 'day', 'date1', 'point']]

In [11]:
filt_lead_df = filt_lead_df.reset_index(drop=True)

In [12]:
unique_cells = air_stagnation.drop_duplicates(subset="cell_point", keep='first',).copy()

In [13]:
unique_cells.head()

,long,lat,stag_days,percent_stag_days,year,month,cell_point
0,-124.75,48.50,0,0.0,1973,1,"(48.5, -124.75)"
1,-124.75,48.25,0,0.0,1973,1,"(48.25, -124.75)"
2,-124.75,48.00,0,0.0,1973,1,"(48.0, -124.75)"
3,-124.75,47.75,0,0.0,1973,1,"(47.75, -124.75)"
4,-124.50,48.50,0,0.0,1973,1,"(48.5, -124.5)"


In [14]:
# Find closest point from a list of points
def closest_point(cell_point, monitor_points, monitor_ids):
    # finds distance between each track and every monitoring station
    dist = []
    for p in monitor_points:
        x = round(haversine(cell_point, p), 4)
        dist.append(x)
    
    idx = dist.index(min(dist))
    
    nearest_monitor = monitor_points[idx]
    distance = dist[idx]
    monitorID = monitor_ids[idx]
    return nearest_monitor, distance, monitorID

In [15]:
monitor_points = list(filt_lead_df['point'])
monitor_ids = list(filt_lead_df['monitorID'])

In [16]:
unique_cell_points = set(unique_cells['cell_point'])
unique_cells['closest_monitor'] = [closest_point(x, monitor_points, monitor_ids) for x in tqdm(unique_cell_points)]

In [17]:
unique_cells['nearest_monitor'] = unique_cells["closest_monitor"].apply(lambda x: x[2])
unique_cells['distance (km)'] = unique_cells["closest_monitor"].apply(lambda x: x[1])
unique_cells['monitor_coord'] = unique_cells["closest_monitor"].apply(lambda x: x[0])
unique_cells = unique_cells.drop(columns="closest_monitor").copy()

### In-fill `air_stagnation` with `unique_cell` stats

In [18]:
merge_df = air_stagnation.merge(unique_cells,
                                     how="outer",
                                     on=["cell_point", "lat", "long"])

merge_df = merge_df.drop(columns=[x for x in merge_df.columns if "_y" in x])
merge_df = merge_df.rename(columns={"stag_days_x":"stag_days",
                                    "percent_stag_days_x":"percent_stag_days",
                                    "year_x":"year",
                                    "month_x":"month"})
merge_df.head()

,long,lat,stag_days,percent_stag_days,year,month,cell_point,nearest_monitor,distance (km),monitor_coord
0,-124.75,48.5,0,0.00,1973,1,"(48.5, -124.75)",2144,107.2609,"(43.098919, -75.22506)"
1,-124.75,48.5,0,0.00,1974,1,"(48.5, -124.75)",2144,107.2609,"(43.098919, -75.22506)"
2,-124.75,48.5,0,0.00,1975,1,"(48.5, -124.75)",2144,107.2609,"(43.098919, -75.22506)"
3,-124.75,48.5,1,3.23,1976,1,"(48.5, -124.75)",2144,107.2609,"(43.098919, -75.22506)"
4,-124.75,48.5,7,22.58,1977,1,"(48.5, -124.75)",2144,107.2609,"(43.098919, -75.22506)"


In [19]:
len(merge_df)

8096782

In [20]:
grid_cells_to_monitor = merge_df.sort_values(['year', 'month']).reset_index(drop=True)

In [21]:
grid_cells_to_monitor = grid_cells_to_monitor.rename(columns={'nearest_monitor': 'nearest_monitorID'})

In [22]:
unique_cells['distance (km)'].std()

62.14319593595711

In [23]:
grid_cells_to_monitor.head()

,long,lat,stag_days,percent_stag_days,year,month,cell_point,nearest_monitorID,distance (km),monitor_coord
0,-124.75,48.50,0,0.0,1973,1,"(48.5, -124.75)",2144,107.2609,"(43.098919, -75.22506)"
1,-124.75,48.25,0,0.0,1973,1,"(48.25, -124.75)",421,111.2752,"(44.989216, -74.712128)"
2,-124.75,48.00,0,0.0,1973,1,"(48.0, -124.75)",451,98.3872,"(44.478939000000004, -73.211517)"
3,-124.75,47.75,0,0.0,1973,1,"(47.75, -124.75)",451,82.3502,"(44.478939000000004, -73.211517)"
4,-124.50,48.50,0,0.0,1973,1,"(48.5, -124.5)",2224,111.3323,"(47.87722, -95.012222)"


In [24]:
grid_cells_to_monitor.to_csv('grid_cells_to_monitor.csv')

### Calculate the Mean Pb Levels for Each Monitor by Month-Year

In [25]:
month_year_pb = daily_lead_df.groupby(['monitorID', 'month','year'], as_index=False)['Pb_mean'].mean()

In [26]:
month_year_pb

,monitorID,month,year,Pb_mean
0,1,1,2010,0.005000
1,1,2,2010,0.002444
2,1,3,2010,0.002375
3,1,6,2011,0.002857
4,1,7,2011,0.001100
...,...,...,...,...
176598,2459,12,2014,0.003767
176599,2459,12,2015,0.004540
176600,2459,12,2016,0.006020
176601,2459,12,2017,0.006180


### air stagnation dataframe with each 2.5 km^2 grid cell linked to its closest monitor, along with the distance to this closest monitor, and the monitor's average lead level for each month-year pair

In [27]:
merged_pb = (month_year_pb.merge(grid_cells_to_monitor, left_on=['monitorID', 'month','year'], 
                                    right_on=['nearest_monitorID', 'month', 'year']))

In [28]:
grid_cells_monthly_pb = (merged_pb.sort_values(['year','month'])
                   #         .drop(columns='nearest_monitorID')
                            .reset_index(drop=True))

In [29]:
grid_cells_monthly_pb.head()

,monitorID,month,year,Pb_mean,long,lat,stag_days,percent_stag_days,cell_point,nearest_monitorID,distance (km),monitor_coord
0,3,1,1980,0.34,-110.25,35.75,2,6.45,"(35.75, -110.25)",3,37.2420,"(32.833881, -109.71861000000001)"
1,3,1,1980,0.34,-110.25,35.50,2,6.45,"(35.5, -110.25)",3,42.3920,"(32.833881, -109.71861000000001)"
2,3,1,1980,0.34,-110.25,35.25,2,6.45,"(35.25, -110.25)",3,57.4670,"(32.833881, -109.71861000000001)"
3,3,1,1980,0.34,-103.50,29.00,2,6.45,"(29.0, -103.5)",3,53.1873,"(32.833881, -109.71861000000001)"
4,3,1,1980,0.34,-100.75,31.00,2,6.45,"(31.0, -100.75)",3,45.5206,"(32.833881, -109.71861000000001)"


In [30]:
grid_cells_monthly_pb['distance (km)'].mean()

76.97278838202618

In [31]:
grid_cells_monthly_pb['distance (km)'].min()

0.2997

In [32]:
grid_cells_monthly_pb['Pb_mean'].min()

-0.00220000002

In [33]:
grid_cells_monthly_pb.to_csv('grid_cells_monthly_pb.csv')